In [39]:
import kfp.components as kfplc
import kfp,os
from kfp.components._structures import MetadataSpec
from typing import Callable
from typing import NamedTuple
import string,random,tempfile,datetime
from dkube.sdk import *

In [40]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)

In [41]:
def visualize(user:str, token:str, run_details: str) -> NamedTuple('Outputs', [
  ('mlpipeline_metrics', 'Metrics'),
]):
    import json
    import pandas as pd
    from minio import Minio
    from dkube.sdk.api import DkubeApi

    print(run_details)
    details = json.loads(run_details)
    jobname = details['Jobname']
    api = DkubeApi(token=token)
    lineage = api.get_training_run_lineage(user, jobname)

    print(lineage)
    model = lineage['run']['parameters']['training']['datums']['outputs'][0]['name']
    version = lineage['run']['parameters']['training']['datums']['outputs'][0]['version']

    model = model.split(':')[1]
    metrics_url = "/users/{}/model/{}/{}/data/metrics/metrics.csv".format(user, model, version)
    print(metrics_url)

    client = Minio(
        "dkube-minio-server.dkube-infra:9000",
        access_key="dkube",
        secret_key="l06dands19s",
        secure=False
    )

    obj = client.get_object(
        "dkube",
        metrics_url,
    )

    metrics = []
    df = pd.read_csv(obj)
    for (name, value) in df.iteritems():
        print(name)
        print(value.values[0])
        metric = {"name": name.strip(), "numberValue": value.values[0]}
        metrics.append(metric)

    metrics = {'metrics': metrics}
    print(metrics)

    return [json.dumps(metrics)]

In [42]:
def componentize(fn: Callable, name: str, desc: str,
                 image: str, annotations: dict, labels: dict):
    labels.update({'wfid': '{{workflow.uid}}', 'runid': '{{pod.name}}'})
    md = MetadataSpec(annotations=annotations, labels=labels)

    fn._component_human_name = name
    fn._component_description = desc

    cfunc = kfp.components.create_component_from_func(
        fn,
        base_image=image,
    )
    cfunc.component_spec.metadata = md
    envs: Mapping[str, str] = {'pipeline': 'true',
                               'wfid': '{{workflow.uid}}', 'runid': '{{pod.name}}'}
    cfunc.component_spec.implementation.container.env = envs
    cfunc.component_spec.save("metricviz.yaml")
    return cfunc

In [43]:
dkube_metrics_vizop = componentize(visualize,
                                 "dkube_metricsviz",
                                 "Visualizer for dkube metrics",
                                 "ocdr/dkube_launcher:viz",
                                 {
                                     'platform': 'Dkube'
                                 },
                                 {
                                     'platform': 'Dkube',
                                     'logger': 'dkubepl',
                                     'dkube.garbagecollect': 'false',
                                     'dkube.garbagecollect.policy':
                                     'all'
                                 })

In [44]:
dkube_metrics_vizop  = kfp.components.load_component_from_file("metricviz.yaml")

client = kfp.Client(existing_token=token)

In [45]:
@kfp.dsl.pipeline(
    name='metric-pl',
    description='csv-pl-metrics'
)
def metrics_pipeline(token):
    
    metrics_pl  = dkube_metrics_vizop(username, token,"W3Omuu")

In [46]:
client.create_run_from_pipeline_func(metrics_pipeline, arguments={"token":token})

RunPipelineResult(run_id=67ac3170-84aa-4ab0-89e7-8fd9b27b12c4)

In [47]:
def generate(name):
    timestamp = datetime.datetime.now().strftime("%H%M%S")
    return "{}-{}{}".format(
        name, timestamp, "".join([random.choice(string.digits) for n in range(4)])
    )
name = generate("metrics.pl")
f = tempfile.NamedTemporaryFile(suffix=".zip", delete=False)
kfp.compiler.Compiler().compile(metrics_pipeline, f.name)
client.upload_pipeline(f.name, pipeline_name=name)

{'created_at': datetime.datetime(2021, 5, 31, 12, 10, 40, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2021, 5, 31, 12, 10, 40, tzinfo=tzlocal()),
                     'id': '1e0983f3-08a9-4113-bad5-f3ab90f7a3c0',
                     'name': 'metrics.pl-1210407374',
                     'package_url': None,
                     'parameters': [{'name': 'token', 'value': None}],
                     'resource_references': [{'key': {'id': '1e0983f3-08a9-4113-bad5-f3ab90f7a3c0',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': '1e0983f3-08a9-4113-bad5-f3ab90f7a3c0',
 'name': 'metrics.pl-1210407374',
 'parameters': [{'name': 'token', 'value': None}],
 'resource_references': None,
 'url': None}